# Presentacion y workshop para analysar RNA-seq de Lonicera Japonica (Honeysuckle)

2. [The honeysuckle genome provides insight into the molecular mechanism of carotenoid metabolism underlying dynamic flower coloration](https://nph.onlinelibrary.wiley.com/doi/10.1111/nph.16552)



1. Preparar la data
2. Quantificar expression
3. Encontrar genes expresados differentemente
4. Explorar los genes
5. Direciones despues
   

In [ ]:
%%bash
#Get some RNA-seq from 2 RNA-seq in PRJNA813701
# Agarrar data de un database conteniendo la majoridad de shotgun-sequencing
#Golden flower stage (SRX14408205,SRX14408206,SRX14408207) 
wget -O rep1_golden_r1.fq.gz  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR182/022/SRR18269922/SRR18269922_1.fastq.gz
wget -O rep1_golden_r2.fq.gz  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR182/022/SRR18269922/SRR18269922_2.fastq.gz


In [ ]:
%%bash
SAMPLE_NAME="rep1_golden"
IN_FOLDER="/data/Epigenetics_Workshop/input_data/"
OUT_FOLDER="/data/Epigenetics_Workshop/input_data/"

# Fastp will trim, QC and toss bad reads 
# Fastp saca "reads" de calidad mala y produce un reporte
fastp -w 16 -i ${IN_FOLDER}/${SAMPLE_NAME}_r1.fq.gz \
      -I ${IN_FOLDER}/${SAMPLE_NAME}_r2.fq.gz \
      -o ${OUT_FOLDER}/${SAMPLE_NAME}_r1_filt.fq.gz \
      -O ${OUT_FOLDER}/${SAMPLE_NAME}_r2_filt.fq.gz \
      -h ${OUT_FOLDER}/${SAMPLE_NAME}_filt.html \
      -j ${OUT_FOLDER}/${SAMPLE_NAME}_filt.json 


In [ ]:
%%bash
# Quantify using kallisto all samples
#Quantify golden flower stage
SAMPLE_NAME="rep1_golden"
IN_FOLDER="/data/Epigenetics_Workshop/input_data/"
cd ${IN_FOLDER}

kallisto quant -t 8 -i lonicera_japonica_kallisto.idx \
    -o lonicera_japonica_${SAMPLE_NAME} --plaintext \
    ${SAMPLE_NAME}_r1_filt.fq.gz ${SAMPLE_NAME}_r2_filt.fq.gz


In [ ]:
#Align reads to transcriptome using STAR for one sample to viz
IX_DIR="/data/Epigenetics_Workshop/input_data/lonicera_japonica_star"
IN_DIR="/data/Epigenetics_Workshop/input_data/"
OUT_DIR="/data/Epigenetics_Workshop/input_data/"
TEMP_DIR="/data/Epigenetics_Workshop/input_data/tmp1"
SAMPLE_NAME="rep1_golden"
cd "${IN_DIR}"

#gunzip if needed
gunzip -c ${SAMPLE_NAME}_r1_filt.fq.gz > ${SAMPLE_NAME}_r1_filt.fq
gunzip -c ${SAMPLE_NAME}_r2_filt.fq.gz > ${SAMPLE_NAME}_r2_filt.fq

#No unzipping because STAR mad
# Currently errors out on laptop, not enough ram
STAR --runMode alignReads \
     --genomeDir $IX_DIR \
     --outSAMtype BAM SortedByCoordinate --runThreadN 2 \
     --outFileNamePrefix $OUT_DIR"/${SAMPLE_NAME}_star/" \
     --outTmpDir $TEMP_DIR \
     --outWigType wiggle \
     --quantMode TranscriptomeSAM GeneCounts \
     --readFilesIn ${SAMPLE_NAME}_r1_filt.fq ${SAMPLE_NAME}_r2_filt.fq


In [ ]:
#Flip output wig to bigwig for viz
CHROM_SIZES="lonicera_japonica_rename_sizes.genome"
IN_DIR="/data/Epigenetics_Workshop/input_data/"
SAMPLE_NAME="rep1_golden"

cd "${IN_DIR}/${SAMPLE_NAME}_star"
wigToBigWig Signal.UniqueMultiple.str1.out.wig \
    ${IN_DIR}/${CHROM_SIZES} \
    ${SAMPLE_NAME}_lonicera_japonica_pos.bw
wigToBigWig Signal.UniqueMultiple.str2.out.wig \
    ${IN_DIR}/${CHROM_SIZES} \
    ${SAMPLE_NAME}_lonicera_japonica_neg.bw

In [ ]:
import pandas as pd
import os
# Differential expression to show caretonoid upregulation
#Read in abundance estimates for all samples and make count matrix

in_dir = "/data/Epigenetics_Workshop/input_data"
samples = ["rep1_golden", 
           "rep2_golden", 
           "rep3_golden", 
           "rep1_green", 
           "rep2_green", 
           "rep3_green"]
sample_df_list = []
for sample in samples:
    sample_df = pd.read_csv(os.path.join(in_dir,f'lonicera_japonica_{sample}/abundance.tsv'), sep="\t")
    sample_df_list.append(sample_df)

sample_counts = pd.concat([
    sample_df_list[0]['est_counts'].astype(int), 
    sample_df_list[1]['est_counts'].astype(int), 
    sample_df_list[2]['est_counts'].astype(int), 
    sample_df_list[3]['est_counts'].astype(int), 
    sample_df_list[4]['est_counts'].astype(int), 
    sample_df_list[5]['est_counts'].astype(int), 
                          ], axis=1, keys=samples)

sample_counts.index=sample_df_list[0]["target_id"]

In [ ]:
sample_counts = sample_counts[sample_counts.sum(axis = 1) > 0]
sample_counts

In [ ]:
sample_counts = sample_counts.T
metadata = pd.DataFrame(zip(sample_counts.index, ['Golden','Golden','Golden','Green', 'Green', 'Green']),
                        columns = ['Sample', 'Condition'])
metadata = metadata.set_index('Sample')
metadata

In [ ]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

dds = DeseqDataSet(counts=sample_counts,
            metadata=metadata,
            design_factors="Condition")

In [ ]:

dds.deseq2()

In [ ]:
from pydeseq2.default_inference import DefaultInference
inference = DefaultInference(n_cpus=8)
stat_res = DeseqStats(dds, inference=inference, contrast = ('Condition','Golden','Green'))
stat_res.summary()


In [ ]:
res = stat_res.results_df
res

In [ ]:
res = res[res.baseMean >= 10]
res

In [ ]:
sigs = res[(res.padj < 0.005) & (abs(res.log2FoldChange) > 2)]
sigs

In [ ]:
sigs.iloc[1:10].index

In [ ]:
sigs.iloc[1].name.split(".")[0]

In [ ]:
%%bash

#See what annotations exist
grep "EVM0011999.1" "../input_data/lonicera_japonica_pep_annotation.txt"
grep "EVM0018585.2" "../input_data/lonicera_japonica_pep_annotation.txt"
grep "EVM0009674.1" "../input_data/lonicera_japonica_pep_annotation.txt"
grep "EVM0010779.2" "../input_data/lonicera_japonica_pep_annotation.txt"



# Places to go from here:

https://github.com/mousepixels/sanbomics_scripts/blob/main/PyDeseq2_DE_tutorial.ipynb